In [2]:
from rdflib import Graph, URIRef, Literal, Namespace, term
from pathlib import Path
datapath = Path("../master-database-files/master-experimental/physics_ontology/")

In [3]:
from json import dumps
def safeVisNetworkJSONToHTMLFile(jsonData, htmlFilePath):
    with open(htmlFilePath, "w") as htmlFile:
        htmlFile.write(
            f"""
            <!DOCTYPE html>
            <html lang="en-US">
            
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1">
                <title>Inline vis</title>
            </head>
            
            <body>
                <div id="mynetwork" style="width:100vw; height:100vh;"></div>
                <script src=" https://cdn.jsdelivr.net/npm/vis-data@7.1.9/peer/umd/vis-data.min.js "></script>
                <script src=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/peer/umd/vis-network.min.js "></script>
                <link href=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/styles/vis-network.min.css " rel="stylesheet">
                <script>
                    var jsonData = {dumps(jsonData)};
                    // create a network
                    var container = document.getElementById("mynetwork");
                    var options = {{}};
                    var network = new vis.Network(container, jsonData, options);
                </script>
            </body>
            
            </html>
            """
        )

In [4]:
def getAllClasses(graph):
    qres = graph.query(
        """SELECT ?s
           WHERE {
              ?s a owl:Class
              }""")
    classes = []
    for row in qres:
        if type(row.s) == term.BNode:
            continue
        classes.append(str(row.s))
    return classes

def getAllPropertiesBetweenClasses(graph):
    qres = graph.query(
        """SELECT ?subject ?property ?object
           WHERE {
                ?property a owl:ObjectProperty.
                ?property rdfs:domain ?subject.
                ?property rdfs:range ?object.
              }""")
    properties = []
    for row in qres:
        properties.append((str(row.subject), str(row.property), str(row.object)))
    return properties

def getAllSubclassRelationsBetweenClasses(graph):
    qres = graph.query(
        """SELECT ?subclass ?superclass
           WHERE {
                ?subclass rdfs:subClassOf ?superclass.
              }""")
    subclassRelations = []
    for row in qres:
        subclassRelations.append((str(row.subclass), str(row.superclass)))
    return subclassRelations

def getAllNamedIndividuals(graph):
    qres = graph.query(
        """SELECT ?s
           WHERE {
              ?s a owl:NamedIndividual
              }""")
    namedIndividuals = []
    for row in qres:
        if type(row.s) == term.BNode:
            continue
        namedIndividuals.append(str(row.s))
    return namedIndividuals

def getAllConnectionsBetweenNamedIndividuals(graph):
    qres = graph.query(
        """SELECT ?subject ?predicate ?object
           WHERE {
                ?subject ?predicate ?object.
                ?subject a owl:NamedIndividual.
                ?object a owl:NamedIndividual.
              }""")
    connections = []
    for row in qres:
        connections.append((str(row.subject), str(row.predicate), str(row.object)))
    return connections

def getNodeName(graph, iri):
    return iri.replace("/", "#").split("#")[-1]

In [26]:
def visualizeOntology(graph, filename = "output.html", showOnlyLinkedClasses = False, showSubclassRelations = True, highlightedIris = []):
    classes = getAllClasses(graph)
    properties = getAllPropertiesBetweenClasses(graph)
    subclassRelations = getAllSubclassRelationsBetweenClasses(graph) if showSubclassRelations else []
    linkedClasses = set([x[0] for x in properties] + [x[2] for x in properties] + [x[0] for x in subclassRelations] + [x[1] for x in subclassRelations])

    nodes = []
    edges = []
    for class_ in classes:
        if showOnlyLinkedClasses and class_ not in linkedClasses:
            continue
        if any([highlightedIri in class_ for highlightedIri in highlightedIris]):
            nodes.append({"id": class_, "label": getNodeName(graph, class_), "color": {"background": "orange", "border": "red"}})
        else:
            nodes.append({"id": class_, "label": getNodeName(graph, class_), "color": {"background": "lightblue", "border": "blue"}})
    for property_ in properties:
        nodes.append({"id": property_[1], "label": getNodeName(graph, property_[1]), "shape": "box", "color": {"background": "lightgreen", "border": "green"}})
        edges.append({"from": property_[0], "to": property_[1], "color": "green"})
        edges.append({"from": property_[1], "to": property_[2],  "arrows": "to", "color": "green"})
    for subclassRelation in subclassRelations:
        edges.append({"from": subclassRelation[0], "to": subclassRelation[1], "label": "subclassOf", "arrows": "to", "dashes": True, "color": "gray"})
    jsonData = {"nodes": nodes, "edges": edges}
    safeVisNetworkJSONToHTMLFile(jsonData, filename)

In [6]:
def visualizeNamedIndividuals(graph, filename = "output.html"):
    namedIndividuals = getAllNamedIndividuals(graph)
    connections = getAllConnectionsBetweenNamedIndividuals(graph)
    nodes = []
    edges = []
    for namedIndividual in namedIndividuals:
        nodes.append({"id": namedIndividual, "label": getNodeName(graph, namedIndividual)})
    for connection in connections:
        edges.append({"from": connection[0], "to": connection[2], "label": getNodeName(graph, connection[1]), "arrows": "to"})
    visjsjson = {"nodes": nodes, "edges": edges}
    safeVisNetworkJSONToHTMLFile(visjsjson, filename)

In [7]:
def listAllIndividualsPerClass(graph, showOnlyLinkedClasses = False):
    classes = getAllClasses(graph)
    properties = getAllPropertiesBetweenClasses(graph)
    linkedClasses = set([x[0] for x in properties] + [x[2] for x in properties])
    individualsPerClass = []
    for class_ in classes:
        if showOnlyLinkedClasses and class_ not in linkedClasses:
            continue
        qres = graph.query(
            f"""SELECT ?s
               WHERE {{
                  ?s a <{class_}>
                  }}""")
        individuals = []
        for row in qres:
            if type(row.s) == term.BNode:
                continue
            individuals.append(str(row.s))
        individualsPerClass.append((class_, individuals))
    # Sort by number of individuals
    individualsPerClass.sort(key=lambda x: len(x[1]), reverse=True)
    for class_, individuals in individualsPerClass:
        print(f"{getNodeName(graph, class_)}: {len(individuals)}")
        for individual in individuals:
            print(f"  - {getNodeName(graph, individual)}")

In [8]:
def countNamedIndividuals(graph):
    namedIndividuals = getAllNamedIndividuals(graph)
    return len(namedIndividuals)

In [9]:
def countRelationsBetweenNamedIndividuals(graph):
    connections = getAllConnectionsBetweenNamedIndividuals(graph)
    return len(connections)

In [6]:
physciGraph = Graph()
physciGraph.parse(str(datapath / "physci.nt"))

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x7f35886a4e00>
Traceback (most recent call last):
  File "/home/gratach/Dokumente/vscode/venv/lib/python3.11/site-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
  File "/home/gratach/Dokumente/vscode/venv/lib/python3.11/site-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '2.0'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x7f35886a4e00>
Traceback (most recent call last):
  File "/home/gratach/Dokumente/vscode/venv/lib/python3.11/site-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  #

<Graph identifier=Nbe3026a2e38c4385a1f2d3883638a458 (<class 'rdflib.graph.Graph'>)>

In [23]:
cPhyOntoGraph = Graph()
cPhyOntoGraph.parse(str(datapath / "cPhyOnto.ttl"), format="turtle")

<Graph identifier=Nd34ed137edf7448bb27baf8009b82db4 (<class 'rdflib.graph.Graph'>)>

In [31]:
visualizeOntology(physciGraph, "output.html", showOnlyLinkedClasses = True, showSubclassRelations = False, highlightedIris = ["https://w3id.org/skgo/physci"])

In [27]:
visualizeOntology(cPhyOntoGraph, "output.html", showSubclassRelations = False)

In [7]:
visualizeNamedIndividuals(physciGraph, "output.html")

In [27]:
listAllIndividualsPerClass(physciGraph, showOnlyLinkedClasses = True)

Equation: 13
  - Conservation_Equations
  - Einstein_Equations
  - Field_equations
  - Infinitesimal_parallel_translations
  - Lagrangian_equation
  - The_wave_equation
  - field_equations_boundary
  - field_equations_with_homogeneous_gravitational_field
  - symmetric_solutions_of_the_equations
  - the_differentieal_equations
  - the_gravitational_equations
  - thermodynamic_expression
  - wave_equations
ResearchWork: 12
  - Binary_Black_Hole_Merger
  - Black_Holes_Entropy
  - First_M87_Event_Horizon_Telescope_Results-I-The_Shadow_of_the_Supermassive_Black_Hole
  - Gravitation_the_only_force_in__Neutrinos
  - Gravitational_Contraction
  - Gravitational_Field_of_a_Mass_Point
  - Implication_of_Gravitational_wave
  - PCA_for_Black_Hole_Images
  - Particle_Creation
  - Particle_problem_in_the_Theory_of_Relativity
  - Relativistic_theory_of_gravitation
  - The_Energy_Distribution_in_the_Normal_Spectrum
Solution: 12
  - A_Special_Kind_of_Singularity_and_Removal
  - Combined_field_electricit

In [38]:
countNamedIndividuals(physciGraph)

193

In [43]:
countRelationsBetweenNamedIndividuals(physciGraph)

179

In [36]:
generatedOnto = Graph()
generatedOnto.parse(str(datapath / ".." / "generated_ontology" / "researchAreaSeeded" / "ontology.ttl"), format="turtle")

<Graph identifier=N4ce53ed8b7104b6b8ffddf70d079879c (<class 'rdflib.graph.Graph'>)>

In [37]:
visualizeOntology(generatedOnto, "output.html", showSubclassRelations = False)